In [8]:
from qiskit.transpiler import CouplingMap
from mirror_gates.pass_managers import Mirage, QiskitLevel3
from mirror_gates.logging import transpile_benchy_logger

In [9]:
from transpile_benchy.library import CircuitLibrary

library = CircuitLibrary.from_txt("../../circuits/iters_select.txt")

In [10]:
coupling_map = CouplingMap.from_line(64)
# coupling_map = CouplingMap.from_heavy_hex(5)

# XXX we hardcoded layout to not be parallelized to avoid a pipe error
# before collecting data for this result -> change the hardcoded value

# maybe requires downgrading to 3.9 to get this to work....????

transpilers = [
    QiskitLevel3(coupling_map, python_sabre=True),
    Mirage(
        coupling_map,
        use_fast_settings=True,
        name="Mirage",
        parallel=True,
        swap_trials=6,
        layout_trials=6,
    ),
]

In [11]:
from transpile_benchy.benchmark import Benchmark

# only interested in TimeMetric, is there by default
benchmark = Benchmark(
    transpilers=transpilers,
    circuit_library=library,
    num_runs=1,
    logger=transpile_benchy_logger,
)
benchmark.run()

INFO:transpile_benchy:Running benchmarks for circuits...
Circuits from library:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:transpile_benchy:Running benchmark for circuit qft_n18


Loading qft_n18 from QASMBench


Circuits from library: 100%|██████████| 1/1 [00:52<00:00, 52.60s/it]


In [12]:
benchmark.summary_statistics(transpilers[0], transpilers[1])

{'total_runtime': {'average_change': 264.56355273362436,
  'aggregrate_change': 264.56355273362436,
  'best_circuit': 'qft_n18',
  'worst_circuit': 'qft_n18'}}

In [15]:
from transpile_benchy.render import plot_benchmark

plot_benchmark(
    benchmark,
    save=1,
    legend_show=1,
    filename="speed",
    plot_type="trendline",
    color_override=[0, 3],
)

/home/evm9/mirror-gates/.venv/lib/python3.11/site-packages/numpy/lib/polynomial.py:667: RuntimeWarning: invalid value encountered in divide
  lhs /= scale


SystemError: <class 'numpy.linalg.LinAlgError'> returned a result with an exception set

In [16]:
print(benchmark)


Transpiler: Mirage-$\sqrt{\texttt{iSWAP}}$

  Metric: total_runtime
  Circuit: qft_n18                                  Mean result: 41.222                         Trials: [41.222068071365356]

Transpiler: Qiskit-$\sqrt{\texttt{iSWAP}}$
  Circuit: qft_n18                                  Mean result: 11.307                         Trials: [11.30723786354065]
